This notebook detects 2 class objects.
- class1: helmet without impact
- class2: helmet with impact

Object Detection part is based on [EfficientDet notebook](https://www.kaggle.com/shonenkov/training-efficientdet) for [global wheat detection competition](https://www.kaggle.com/c/global-wheat-detection) by [shonenkov](https://www.kaggle.com/shonenkov), which is using [github repos efficientdet-pytorch](https://github.com/rwightman/efficientdet-pytorch) by [@rwightman](https://www.kaggle.com/rwightman).

Inference part can be foud [here](https://www.kaggle.com/its7171/2class-object-detection-inference/).

In [1]:
Kaggle = False
Colab = !Kaggle

In [2]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"
    os.chdir(path)
    os.listdir(path)

Mounted at /content/drive


In [3]:
if Kaggle:
    !pip install ../input/nfl-lib/timm-0.1.26-py3-none-any.whl
    !tar xfz ../input/nfl-lib/pkgs.tgz
else:
    !pip install NFL/nfl-lib/timm-0.1.26-py3-none-any.whl
    !tar xfz NFL/nfl-lib/pkgs.tgz    
    !pip install albumentations==0.4.6
# for pytorch1.6
cmd = "sed -i -e 's/ \/ / \/\/ /' timm-efficientdet-pytorch/effdet/bench.py"
!$cmd

Processing ./NFL/nfl-lib/timm-0.1.26-py3-none-any.whl
     |████████████████████████████████| 122kB 5.7MB/s 
     |████████████████████████████████| 952kB 18.9MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=a4ada8700506022abe4364c810e9f35f2f66bef7e83925b309f870aa2fc0d04e
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [4]:
import sys
sys.path.insert(0, "timm-efficientdet-pytorch")
sys.path.insert(0, "omegaconf")

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pandas as pd
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from tqdm import tqdm

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

# Data Preparation

In [5]:
if Kaggle:
    BASEPATH = "../input/nfl-impact-detection"
    outdir = '.'
    TRAIN_IMGPATH = outdir
else:
    PATH = 'NFL/'
    BASEPATH = PATH + 'Data'
    TRAIN_IMGPATH = BASEPATH
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/efficientDet-res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    MODELNAME = "ED5-512"
    VERSION = '{}'.format(MODELNAME)
    outdir = os.path.join(outdir, VERSION)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    # from datetime import datetime, timedelta
    # dateTimeObj = datetime.now()
    # timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")
    # modelpath = os.path.join(outdir, 'all-kfold-hairaug-456-norm-metanew-b16')
    timestampStr = 'org-epoch20-aug1'
    outdir = os.path.join(outdir, timestampStr)
    if not os.path.exists(outdir):
        os.mkdir(outdir)  

In [6]:
# video_labels = pd.read_csv(os.path.join(BASEPATH, 'train_labels.csv')).fillna(0)
# video_labels_with_impact = video_labels[video_labels['impact'] > 0]
# for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
#     frames = np.array([-4,-3,-2,-1,1,2,3,4])+row[1]
#     video_labels.loc[(video_labels['video'] == row[0]) 
#                                  & (video_labels['frame'].isin(frames))
#                                  & (video_labels['label'] == row[2]), 'impact'] = 1
# video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str) + '.png'
# video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
# video_labels['impact'] = video_labels['impact'].astype(int)+1
# video_labels['x'] = video_labels['left']
# video_labels['y'] = video_labels['top']
# video_labels['w'] = video_labels['width']
# video_labels['h'] = video_labels['height']
# video_labels.head()

video_labels = pd.read_csv(os.path.join(BASEPATH,'video_labels.csv'), index_col=0)
display(video_labels.head())
print(video_labels.shape)

,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility,image_name,x,y,w,h
0,57583,82,Endzone,57583_000082_Endzone.mp4,34,V73,655,21,331,15,1,0,0.0,0.0,57583_000082_Endzone_34.png,655,331,21,15
1,57583,82,Endzone,57583_000082_Endzone.mp4,34,H99,583,21,312,30,2,0,0.0,0.0,57583_000082_Endzone_34.png,583,312,21,30
2,57583,82,Endzone,57583_000082_Endzone.mp4,34,V15,1069,22,301,20,1,0,0.0,0.0,57583_000082_Endzone_34.png,1069,301,22,20
3,57583,82,Endzone,57583_000082_Endzone.mp4,34,H97,402,21,313,29,1,0,0.0,0.0,57583_000082_Endzone_34.png,402,313,21,29
4,57583,82,Endzone,57583_000082_Endzone.mp4,34,V72,445,21,328,16,1,0,0.0,0.0,57583_000082_Endzone_34.png,445,328,21,16


(197838, 19)


In [7]:
# display(video_labels.head())
# print(video_labels.shape)
# video_labels.to_csv(os.path.join(BASEPATH,'video_labels.csv'), index=False)

In [8]:
np.random.seed(0)
video_names = np.random.permutation(video_labels.video.unique())
valid_video_len = int(len(video_names)*0.2)
video_valid = video_names[:valid_video_len]
video_train = video_names[valid_video_len:]
images_valid = video_labels[ video_labels.video.isin(video_valid)].image_name.unique()
images_train = video_labels[~video_labels.video.isin(video_valid)].image_name.unique()

In [9]:
def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
    video_path=f"{video_dir}/{video_name}"
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)
    if only_with_impact:
        boxes_all = video_labels.query("video == @video_name")
        print(video_path, boxes_all[boxes_all.impact > 1.0].shape[0])
    else:
        print(video_path)
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        frame += 1
        if only_with_impact:
            boxes = video_labels.query("video == @video_name and frame == @frame")
            boxes_with_impact = boxes[boxes.impact > 1.0]
            if boxes_with_impact.shape[0] == 0:
                continue
        img_name = f"{video_name}_frame{frame}"
        image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{frame}.png')
        _ = cv2.imwrite(image_path, img)

In [10]:
uniq_video = video_labels.video.unique()
# video_dir = '/kaggle/input/nfl-impact-detection/train'
video_dir = os.path.join(BASEPATH, 'train')
out_dir = os.path.join(TRAIN_IMGPATH, 'train_images')
print(out_dir)
# !mkdir -p $out_dir
# for video_name in uniq_video:
#     mk_images(video_name, video_labels, video_dir, out_dir)

NFL/Data/train_images


## Albumentations

In [11]:
def get_train_transforms():
    return A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, interpolation=1, border_mode=4, 
                               value=None, mask_value=None, always_apply=False, p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Blur(blur_limit=5, always_apply=False, p=0.1),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.1),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

## Dataset

In [12]:
TRAIN_ROOT_PATH = os.path.join(TRAIN_IMGPATH, 'train_images')

class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()

        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        image, boxes, labels = self.load_image_and_boxes(index)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = torch.tensor(labels)
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    break
        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        # print(f'{TRAIN_ROOT_PATH}/{image_id}')
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
        # img = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR)
        # if img is NoneType:
        #     print(f'{TRAIN_ROOT_PATH}/{image_id}')
        # image = img.copy().astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_name'] == image_id]
        boxes = records[['x', 'y', 'w', 'h']].values
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        labels = records['impact'].values
        return image, boxes, labels

In [13]:
train_dataset = DatasetRetriever(
    image_ids=images_train,
    marking=video_labels,
    transforms=get_train_transforms(),
    test=False,
)

validation_dataset = DatasetRetriever(
    image_ids=images_valid,
    marking=video_labels,
    transforms=get_valid_transforms(),
    test=True,
)

## Fitter

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [15]:
import warnings

warnings.filterwarnings("ignore")

class Fitter:
    
    def __init__(self, model, device, config):
        self.config = config
        self.epoch = 0

        self.base_dir = f'./{config.folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)
        
        self.log_path = f'{self.base_dir}/log.txt'
        self.best_summary_loss = 10**5

        self.model = model
        self.device = device

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')

            t = time.time()
            summary_loss = self.train_one_epoch(train_loader)

            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            self.save(f'{self.base_dir}/last-checkpoint.bin')

            t = time.time()
            summary_loss = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_summary_loss:
                self.best_summary_loss = summary_loss.avg
                self.model.eval()
                self.save(f'{self.base_dir}/best-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-checkpoint-*epoch.bin'))[:-3]:
                    os.remove(path)

            if self.config.validation_scheduler:
                self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images = torch.stack(images)
                batch_size = images.shape[0]
                images = images.to(self.device).float()
                boxes = [target['boxes'].to(self.device).float() for target in targets]
                labels = [target['labels'].to(self.device).float() for target in targets]

                loss, _, _ = self.model(images, boxes, labels)
                summary_loss.update(loss.detach().item(), batch_size)

        return summary_loss

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        t = time.time()
        for step, (images, targets, image_ids) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            
            images = torch.stack(images)
            images = images.to(self.device).float()
            batch_size = images.shape[0]
            boxes = [target['boxes'].to(self.device).float() for target in targets]
            labels = [target['labels'].to(self.device).float() for target in targets]

            self.optimizer.zero_grad()
            
            loss, _, _ = self.model(images, boxes, labels)
            
            loss.backward()

            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss
    
    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_summary_loss': self.best_summary_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_summary_loss = checkpoint['best_summary_loss']
        self.epoch = checkpoint['epoch'] + 1
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [16]:
class TrainGlobalConfig:
    num_workers = 4
    batch_size = 4 
    n_epochs = 20
    lr = 0.0002
    folder = os.path.join(outdir, 'effdet5-models')
    verbose = True
    verbose_step = 1
    step_scheduler = False
    validation_scheduler = True
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )

In [17]:
def collate_fn(batch):
    return tuple(zip(*batch))

def run_training():
    if Kaggle:
        device = torch.device('cuda:0')
    else:
        device = ('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        sampler=RandomSampler(train_dataset),
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
        collate_fn=collate_fn,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )

    fitter = Fitter(model=net, device=device, config=TrainGlobalConfig)
    fitter.fit(train_loader, val_loader)

In [18]:
def get_net():
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    checkpoint = torch.load(os.path.join(PATH, 'nfl-models/efficientdet_d5-ef44aea8.pth'))
    net.load_state_dict(checkpoint)
    config.num_classes = 2
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    return DetBenchTrain(net, config)

net = get_net()

In [ ]:
run_training()

Fitter prepared. Device is cuda

2020-12-05T18:51:18.491533
LR: 0.0002
[RESULT]: Train. Epoch: 0, summary_loss: 34.96576, time: 3210.38497
[RESULT]: Val. Epoch: 0, summary_loss: 0.75771, time: 748.97816

2020-12-05T19:57:22.463525
LR: 0.0002
[RESULT]: Train. Epoch: 1, summary_loss: 0.68814, time: 1870.60920
[RESULT]: Val. Epoch: 1, summary_loss: 0.58253, time: 110.45883

2020-12-05T20:30:29.274830
LR: 0.0002
[RESULT]: Train. Epoch: 2, summary_loss: 0.55769, time: 1882.27487
[RESULT]: Val. Epoch: 2, summary_loss: 0.48443, time: 109.43748

2020-12-05T21:03:45.121714
LR: 0.0002
[RESULT]: Train. Epoch: 3, summary_loss: 0.50383, time: 1862.00347
[RESULT]: Val. Epoch: 3, summary_loss: 0.47818, time: 105.88258

2020-12-05T21:36:37.177517
LR: 0.0002
[RESULT]: Train. Epoch: 4, summary_loss: 0.47522, time: 1853.74262
[RESULT]: Val. Epoch: 4, summary_loss: 0.44413, time: 105.49370

2020-12-05T22:09:20.855155
LR: 0.0002
[RESULT]: Train. Epoch: 5, summary_loss: 0.45104, time: 1831.70156
[RESULT]: V

In [ ]:
# clearing working dir
# be careful when running this code on local environment!
# !rm -rf *
# !mv * /tmp/train_images

# import shutil
# shutil.rmtree(out_dir)  